In [1]:
import os
import pandas as pd
import numpy as np
import re

os.chdir('../..')
pd.set_option('display.max_columns', None)

ONLINE_DATA_PATH = 'data/external/online_fpl_data/2017_18/'

# Check 2018/19 data reliability

Compare data scraped from https://fantasy.premierleague.com/ against data from [GitHub](https://github.com/vaastav/Fantasy-Premier-League/tree/master/data?fbclid=IwAR1-z1MS9faPHz-JHrXQDGqvV3e25Hy6IzGQnnmkkQhoxux3QDRTamrmuZg)

In [2]:
sg_2018_19 = pd.read_csv('data/interim/clean_scraped_2018_19.csv')
print(sg_2018_19.shape)
sg_2018_19.head()

(18654, 29)


,gameweek,points,minutes_played,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bonus_points_system,influence,creativity,threat,ict_index,net_transfers,selected_by,value,name,team,position,opposition,home_or_away,team_goals,opposition_goals
0,1,8,86,1,0,1,0,0,0,0,0,0,0,0,20.0,47.6,29.0,92.0,16.9,0.0,2478960.0,13.0,Salah,LIV,MID,WHU,H,4,0
1,2,9,90,0,2,1,0,0,0,0,0,0,0,0,22.0,30.6,41.5,81.0,15.3,38860.0,2799491.0,13.0,Salah,LIV,MID,CRY,A,2,0
2,3,8,90,1,0,1,0,0,0,0,0,0,0,0,28.0,49.0,72.0,82.0,20.3,5370.0,2892383.0,13.0,Salah,LIV,MID,BHA,H,1,0
3,4,2,70,0,0,0,1,0,0,0,0,0,0,0,7.0,5.0,12.5,23.0,4.1,32547.0,2995784.0,13.0,Salah,LIV,MID,LEI,A,2,1
4,5,2,90,0,0,0,1,0,0,0,0,0,0,0,7.0,16.0,31.8,74.0,12.2,-261759.0,2754458.0,13.0,Salah,LIV,MID,TOT,A,2,1


In [3]:
github_2018_19 = pd.read_csv('data/external/online_fpl_data/2018_19/merged_gw.csv', encoding = "ISO-8859-1")
print(github_2018_19.shape)
github_2018_19.head()

(21790, 56)


,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW
0,Aaron_Cresswell_402,0,0,0,0,0,0,0,0,0,0.0,0,0,402,0,0,5,0,0,0,0.0,402,0.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,0,0,0,12,0,0,0,0,0,0,1,0,103396,0,0,0,0,4,0.0,0,0,0,0,55,False,0,0,1
1,Aaron_Lennon_83,0,22,0,1,0,6,1,1,17,12.3,0,0,83,0,0,8,1,0,0,3.9,83,10.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,90,0,1,16,0,0,0,0,2,0,1,0,15138,1,2,0,0,0,17.0,3,0,0,0,50,False,0,0,1
2,Aaron_Mooy_199,0,51,0,0,0,24,0,2,40,18.2,1,0,199,0,0,4,1,3,0,3.8,199,20.2,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,6,0,0,0,0,11,0,1,0,192110,1,6,0,3,0,0.0,2,0,0,0,55,True,0,0,1
3,Aaron_Ramsey_14,0,11,0,0,0,7,0,0,7,10.8,1,0,14,0,0,1,0,1,0,2.9,14,9.4,1,2018-08-12T15:00:00Z,12 Aug 16:00,0,0,53,2,0,13,0,0,0,0,1,0,1,0,60423,0,2,0,2,0,9.0,1,0,0,0,75,True,0,0,1
4,Aaron_Wan-Bissaka_145,1,29,1,0,3,38,1,11,19,14.0,2,0,145,0,0,3,0,0,0,6.0,145,46.0,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,9,0,0,0,0,7,0,1,0,652304,2,0,0,2,0,0.0,12,0,0,0,40,False,0,0,1


+3000 more rows and 27 more columns in GitHub data!

In [5]:
# No team in GitHub data so try joining on Name-HomeOrAway-NetTransfers

sg_2018_19['join-key'] = sg_2018_19['name'] + '-' + sg_2018_19['home_or_away'] + '-' + sg_2018_19['net_transfers'].astype(str)

github_2018_19['name'] = github_2018_19['name'].str.split('_', expand=True)[1]
github_2018_19['was_home'] = github_2018_19['was_home'].replace({False:'A', True:'H'})
github_2018_19['transfers_balance'] = github_2018_19['transfers_balance'].astype(float)
github_2018_19['join-key'] = github_2018_19['name'] + '-' + github_2018_19['was_home'] + '-' + github_2018_19['transfers_balance'].astype(str)

In [9]:
combined = github_2018_19.merge(sg_2018_19, how='left', on='join-key', suffixes=('_github', '_sg'))

In [10]:
combined.head()

,name_github,assists_github,attempted_passes,big_chances_created,big_chances_missed,bonus_github,bps,clean_sheets_github,clearances_blocks_interceptions,completed_passes,creativity_github,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded_github,goals_scored_github,ict_index_github,id,influence_github,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals_github,penalties_conceded,penalties_missed_github,penalties_saved_github,recoveries,red_cards_github,round,saves_github,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat_github,total_points,transfers_balance,transfers_in,transfers_out,value_github,was_home,winning_goals,yellow_cards_github,GW,join-key,gameweek,points,minutes_played,goals_scored_sg,assists_sg,clean_sheets_sg,goals_conceded_sg,own_goals_sg,penalties_saved_sg,penalties_missed_sg,yellow_cards_sg,red_cards_sg,saves_sg,bonus_sg,bonus_points_system,influence_sg,creativity_sg,threat_sg,ict_index_sg,net_transfers,selected_by,value_sg,name_sg,team,position,opposition,home_or_away,team_goals,opposition_goals
0,Cresswell,0,0,0,0,0,0,0,0,0,0.0,0,0,402,0,0,5,0,0,0,0.0,402,0.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,0,0,0,12,0,0,0,0,0,0,1,0,103396,0,0,0,0,4,0.0,0,0.0,0,0,55,A,0,0,1,Cresswell-A-0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103396.0,5.5,Cresswell,WHU,DEF,LIV,A,0.0,4.0
1,Lennon,0,22,0,1,0,6,1,1,17,12.3,0,0,83,0,0,8,1,0,0,3.9,83,10.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,90,0,1,16,0,0,0,0,2,0,1,0,15138,1,2,0,0,0,17.0,3,0.0,0,0,50,A,0,0,1,Lennon-A-0.0,1.0,3.0,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,10.0,12.3,17.0,3.9,0.0,15138.0,5.0,Lennon,BUR,MID,SOU,A,0.0,0.0
2,Mooy,0,51,0,0,0,24,0,2,40,18.2,1,0,199,0,0,4,1,3,0,3.8,199,20.2,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,6,0,0,0,0,11,0,1,0,192110,1,6,0,3,0,0.0,2,0.0,0,0,55,H,0,0,1,Mooy-H-0.0,1.0,2.0,90.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,20.2,18.2,0.0,3.8,0.0,192110.0,5.5,Mooy,HUD,MID,CHE,H,0.0,3.0
3,Ramsey,0,11,0,0,0,7,0,0,7,10.8,1,0,14,0,0,1,0,1,0,2.9,14,9.4,1,2018-08-12T15:00:00Z,12 Aug 16:00,0,0,53,2,0,13,0,0,0,0,1,0,1,0,60423,0,2,0,2,0,9.0,1,0.0,0,0,75,H,0,0,1,Ramsey-H-0.0,1.0,1.0,53.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,9.4,10.8,9.0,2.9,0.0,60423.0,7.5,Ramsey,ARS,MID,MCI,H,0.0,2.0
4,Wan-Bissaka,1,29,1,0,3,38,1,11,19,14.0,2,0,145,0,0,3,0,0,0,6.0,145,46.0,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,9,0,0,0,0,7,0,1,0,652304,2,0,0,2,0,0.0,12,0.0,0,0,40,A,0,0,1,Wan-Bissaka-A-0.0,1.0,12.0,90.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,38.0,46.0,14.0,0.0,6.0,0.0,652304.0,4.0,Wan-Bissaka,CRY,DEF,FUL,A,2.0,0.0


In [11]:
# Players in GitHub data who are not in scraped data
print(combined[combined['gameweek'].isnull()]['name_github'].sort_values().nunique())
combined[combined['gameweek'].isnull()]['name_github'].sort_values().unique()

137


array(['Abraham', 'Agosto Ramírez', 'Anício Caldeira Duarte',
       'Arrizabalaga', 'Bakayoko', 'Baldock', 'Baningime',
       'Barbosa Pereira', 'Batth', 'Benalouane', 'Benkovic', 'Blind',
       'Bogle', 'Bolasie', 'Bonatini', 'Borges Da Silva', 'Boufal',
       'Browning', 'Byram', 'Carrillo', 'Castro Otto', 'Cook',
       'Costa Silva', 'Courtois', 'Cunninghamm', 'Dalot Teixeira',
       'Davis', 'De Gea', 'Defoe', 'Dembélé', 'Diabaté', 'Diaz',
       'Douglas', 'Dowell', 'Enobakhare', 'Fellaini', 'Femenía Far',
       'Fernandes', 'Fernandes da Silva Junior', 'Frello Filho',
       'Fàbregas', 'Gabbiadini', 'Gayle', 'Gonçalves Miranda', 'Halford',
       'Hause', 'Hemed', 'Hernández Balcázar', 'Hoedt', 'Holgate',
       'Hugill', 'Iborra', 'Ince', 'Izquierdo Mena', 'Jach', 'Johansen',
       'Jorgensen', 'Kaboul', 'Kaikai', 'Kamara', 'Karius', 'Ki',
       'Klaassen', 'Klavan', 'Knight', 'Koiki', 'Luiz Moreira Marinho',
       'Luiz Rosa', 'Malone', 'Martina', 'Martínez', 'Mato S

In [12]:
combined = sg_2018_19.merge(github_2018_19, how='left', on='join-key', suffixes=('_sg', '_github'))

In [13]:
# Players in scraped data who are not in GitHub data
print(combined[combined['GW'].isnull()]['name_sg'].sort_values().nunique())
combined[combined['GW'].isnull()]['name_sg'].sort_values().unique()

53


array(['Adam Smith', 'Adrián', 'Alisson', 'André Gomes', 'Bernard',
       'Bernardo', 'Bernardo Silva', 'Brian Murphy', 'Bruno',
       'Carlos Sánchez', 'Chamberlain', 'Chicharito', 'Cunningham',
       'Dalot', 'Danilo', 'David Luiz', 'David Silva', 'Ederson',
       'Emerson', 'Fabinho', 'Fabri', 'Felipe Anderson', 'Fernandinho',
       'Fred', 'Izquierdo', 'Jazz Richards', 'Jesus', 'Jonny', 'Jorginho',
       'Joselu', 'Kenedy', 'Kepa', 'Ki Sung-yueng', 'Kiko Femenía',
       'Lewis Cook', 'Lucas', 'Lucas Moura', 'Moutinho', 'Neves',
       'Oriol Romeu', 'Patrício', 'Pedro', 'Pereira', 'Richarlison',
       'Sokratis', 'Steve Cook', 'Steven Sessegnon', 'Success', 'Vinagre',
       'Willian', 'Xande Silva', 'Zanka', 'de Gea'], dtype=object)

Looks like:

1) Some differences are due to alternative player names e.g. Kepa vs Arrizabalaga

2) Players drop out of FPL website (scraped data) when they move club e.g. Courtois, Gabbiadini

## What are additional columns?

In [14]:
github_2018_19.columns

Index(['name', 'assists', 'attempted_passes', 'big_chances_created',
       'big_chances_missed', 'bonus', 'bps', 'clean_sheets',
       'clearances_blocks_interceptions', 'completed_passes', 'creativity',
       'dribbles', 'ea_index', 'element', 'errors_leading_to_goal',
       'errors_leading_to_goal_attempt', 'fixture', 'fouls', 'goals_conceded',
       'goals_scored', 'ict_index', 'id', 'influence', 'key_passes',
       'kickoff_time', 'kickoff_time_formatted', 'loaned_in', 'loaned_out',
       'minutes', 'offside', 'open_play_crosses', 'opponent_team', 'own_goals',
       'penalties_conceded', 'penalties_missed', 'penalties_saved',
       'recoveries', 'red_cards', 'round', 'saves', 'selected', 'tackled',
       'tackles', 'target_missed', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'winning_goals', 'yellow_cards', 'GW', 'join-key'],
      dtype='object')

In [56]:
common_to_both = [s for s in list(combined.columns) if "_github" in s]
common_to_both = [s.replace('_github', '') for s in common_to_both] + ['join-key']
common_to_both

['name',
 'assists',
 'bonus',
 'clean_sheets',
 'creativity',
 'goals_conceded',
 'goals_scored',
 'ict_index',
 'influence',
 'own_goals',
 'penalties_missed',
 'penalties_saved',
 'red_cards',
 'saves',
 'threat',
 'value',
 'yellow_cards',
 'join-key']

In [40]:
lst = list(set(github_2018_19.columns) - set(common_to_both))
sorted(lst, key=str.lower)

['attempted_passes',
 'big_chances_created',
 'big_chances_missed',
 'bps',
 'clearances_blocks_interceptions',
 'completed_passes',
 'dribbles',
 'ea_index',
 'element',
 'errors_leading_to_goal',
 'errors_leading_to_goal_attempt',
 'fixture',
 'fouls',
 'GW',
 'id',
 'key_passes',
 'kickoff_time',
 'kickoff_time_formatted',
 'loaned_in',
 'loaned_out',
 'minutes',
 'offside',
 'open_play_crosses',
 'opponent_team',
 'penalties_conceded',
 'recoveries',
 'round',
 'selected',
 'tackled',
 'tackles',
 'target_missed',
 'team_a_score',
 'team_h_score',
 'total_points',
 'transfers_balance',
 'transfers_in',
 'transfers_out',
 'was_home',
 'winning_goals']

Lots of player level data which could be quite predictive e.g. recoveries, big_chances_created

## Does column data match?

In [43]:
common_df = sg_2018_19[common_to_both].merge(github_2018_19[common_to_both],
                                             how='inner',
                                             on='join-key',
                                             suffixes=('_sg', '_github'))

In [67]:
common_to_both.append('join-key')

In [72]:
common_to_both.remove('join-key')

In [74]:
for col in common_to_both:
    print(col)
    common_to_both.append('join-key')
    common_df = sg_2018_19[common_to_both].merge(github_2018_19[common_to_both],
                                             how='inner',
                                             on='join-key',
                                             suffixes=('_sg', '_github'))
    common_to_both.remove('join-key')
    subset = common_df[[f'{col}_sg', f'{col}_github']]
    subset['is_different'] = np.where(subset[f'{col}_sg'] == subset[f'{col}_github'],
                                      0,
                                      1)
    print(subset['is_different'].sum())

name


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


0
assists
15
bonus
24
clean_sheets
90
creativity
273
goals_conceded
231
goals_scored
19
ict_index
291
influence
275
own_goals
0
penalties_missed
0
penalties_saved
0
red_cards
2
saves
10
threat
206
value
17499
yellow_cards
33


In [76]:
common_to_both.append('join-key')
common_df = sg_2018_19[common_to_both].merge(github_2018_19[common_to_both],
                                             how='inner',
                                             on='join-key',
                                             suffixes=('_sg', '_github'))

In [79]:
common_df[common_df['assists_sg'] != common_df['assists_github']][['assists_sg', 'assists_github']]

,assists_sg,assists_github
2954,0,1
2955,1,0
4936,0,1
4937,1,0
5458,1,0
6010,1,0
6011,0,1
6190,0,1
6191,1,0
7840,1,0


# Old stuff (TBD):

In [2]:
full_data_sg_17_18 = pd.read_csv('data/interim/clean_scraped_2017_18.csv')
print(full_data_sg_17_18.shape)
full_data_sg_17_18.head()

(18973, 30)


,GW Gameweek,Pts,MPMinutes played,GSGoals scored,AAssists,CSClean sheets,GCGoals conceded,OGOwn goals,PSPenalties saved,PMPenalties missed,YCYellow cards,RCRed cards,SSaves,BBonus,BPSBonus Points System,IInfluence,CCreativity,TThreat,IIICT Index,NT Net Transfers,SB Selected by,£ Value,Name,Team,Position,Total score rank,opposition,home_or_away,team_goals,opposition_goals
0,1,11,85,1,1,0,2,0,0,0,0,0,0,1,26,24.6,2.8,55,8.2,0,874608,9.0,Salah,LIV,MID,1,WAT,A,3,3
1,2,1,29,0,0,0,0,0,0,0,0,0,0,0,0,10.4,12.3,28,5.1,175914,1293309,9.1,Salah,LIV,MID,1,CRY,H,1,0
2,3,11,90,1,1,1,0,0,0,0,0,0,0,0,39,70.4,25.3,103,19.9,-184736,1158692,9.0,Salah,LIV,MID,1,ARS,H,4,0
3,4,1,45,0,0,0,1,0,0,0,0,0,0,0,4,7.8,13.8,25,4.7,177596,1422941,9.1,Salah,LIV,MID,1,MCI,A,0,5
4,5,10,90,1,0,0,1,0,0,0,0,0,0,3,27,51.2,35.8,87,17.4,122769,1571656,9.1,Salah,LIV,MID,1,BUR,H,1,1


In [19]:
full_data_sg_17_18 = pd.read_csv('data/interim/clean_scraped_2017_18.csv')
print(full_data_sg_17_18.shape)
full_data_sg_17_18.head()

(18973, 30)


,GW Gameweek,Pts,MPMinutes played,GSGoals scored,AAssists,CSClean sheets,GCGoals conceded,OGOwn goals,PSPenalties saved,PMPenalties missed,YCYellow cards,RCRed cards,SSaves,BBonus,BPSBonus Points System,IInfluence,CCreativity,TThreat,IIICT Index,NT Net Transfers,SB Selected by,£ Value,Name,Team,Position,Total score rank,opposition,home_or_away,team_goals,opposition_goals
0,1,11,85,1,1,0,2,0,0,0,0,0,0,1,26,24.6,2.8,55,8.2,0,874608,9.0,Salah,LIV,MID,1,WAT,A,3,3
1,2,1,29,0,0,0,0,0,0,0,0,0,0,0,0,10.4,12.3,28,5.1,175914,1293309,9.1,Salah,LIV,MID,1,CRY,H,1,0
2,3,11,90,1,1,1,0,0,0,0,0,0,0,0,39,70.4,25.3,103,19.9,-184736,1158692,9.0,Salah,LIV,MID,1,ARS,H,4,0
3,4,1,45,0,0,0,1,0,0,0,0,0,0,0,4,7.8,13.8,25,4.7,177596,1422941,9.1,Salah,LIV,MID,1,MCI,A,0,5
4,5,10,90,1,0,0,1,0,0,0,0,0,0,3,27,51.2,35.8,87,17.4,122769,1571656,9.1,Salah,LIV,MID,1,BUR,H,1,1


In [20]:
full_data_sg_17_18.groupby(['GW Gameweek']).count()['Pts']

GW Gameweek
1     434
2     436
3     448
4     461
5     463
6     465
7     468
8     471
9     473
10    475
11    477
12    479
13    480
14    481
15    483
16    485
17    486
18    487
19    490
20    491
21    442
22    545
23    502
24    507
25    516
26    528
27    530
28    533
29    536
30    537
31    225
32    540
33    544
34    757
35    336
36    547
37    865
38    550
Name: Pts, dtype: int64

In [21]:
full_data_sg_17_18['Name'].sort_values().unique()

array(['Abraham', 'Adam', 'Adam Smith', 'Adarabioyo', 'Adrián', 'Afellay',
       'Agüero', 'Aké', 'Albrighton', 'Alderweireld', 'Alexander-Arnold',
       'Allen', 'Alli', 'Alonso', 'Amartey', 'Ampadu', 'André Ayew',
       'Antonio', 'Arfield', 'Arnautovic', 'Arter', 'Atsu', 'Aubameyang',
       'Aurier', 'Austin', 'Azpilicueta', 'Bachmann', 'Badou Ndiaye',
       'Bailly', 'Baines', 'Bakayoko', 'Baldock', 'Baningime', 'Bardsley',
       'Barkley', 'Barnes', 'Barry', 'Bartley', 'Bauer', 'Bednarek',
       'Begovic', 'Bellerín', 'Benalouane', 'Benteke', 'Berahino',
       'Bernardo Silva', 'Bertrand', 'Billing', 'Blind', 'Bolasie',
       'Bong', 'Bony', 'Boruc', 'Boufal', 'Brad Smith', 'Brady', 'Bravo',
       'Britos', 'Britton', 'Brown', 'Browne', 'Bruno', 'Brunt', 'Burke',
       'Butcher', 'Butland', 'Byram', 'Caballero', 'Cabaye', 'Cahill',
       'Calvert-Lewin', 'Camacho', 'Cameron', 'Campbell', 'Can', 'Capoue',
       'Carrick', 'Carrillo', 'Carroll', 'Cathcart', 'Cavalieri',

In [22]:
# Combine all GW data:
full_data_online_17_18 = pd.DataFrame()
for gw in range(1,39):
    gw_df = pd.read_csv(ONLINE_DATA_PATH + 'gw{}.csv'.format(gw), encoding = "ISO-8859-1")
    gw_df['Gameweek'] = gw
    full_data_online_17_18 = full_data_online_17_18.append(gw_df,ignore_index=True)
    print(len(gw_df))

511
516
531
544
546
550
554
557
559
563
566
569
570
571
574
577
578
580
585
586
526
651
598
603
613
625
627
630
633
634
267
637
641
877
407
644
1020
647


In [23]:
full_data_online_17_18.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,Gameweek
0,Aaron_Cresswell,0,10,0,0,0,3,0,0,9,0.6,0,0,443,0,0,9,0,2,0,1.9,443,0.4,0,2017-08-13T15:00:00Z,13 Aug 16:00,0,0,9,0,0,12,0,0,0,0,0,0,1,0,25136,0,0,0,0,4,18.0,0,0,0,0,50,False,0,0,1
1,Aaron_Lennon,0,0,0,0,0,0,0,0,0,0.0,0,0,153,0,0,5,0,0,0,0.0,153,0.0,0,2017-08-12T14:00:00Z,12 Aug 15:00,0,0,0,0,0,15,0,0,0,0,0,0,1,0,4681,0,0,0,0,1,0.0,0,0,0,0,55,True,0,0,1
2,Aaron_Mooy,1,52,0,0,0,22,1,5,41,46.9,0,0,172,0,0,4,0,0,0,8.7,172,40.2,3,2017-08-12T14:00:00Z,12 Aug 15:00,0,0,90,0,1,6,0,0,0,0,5,0,1,0,59955,2,0,0,3,0,0.0,6,0,0,0,55,False,0,0,1
3,Aaron_Ramsey,0,11,0,1,0,16,0,0,8,11.2,0,0,18,0,0,1,0,0,1,6.7,18,29.6,1,2017-08-11T18:45:00Z,11 Aug 19:45,0,0,23,0,0,9,0,0,0,0,2,0,1,0,33792,0,0,3,3,4,26.0,6,0,0,0,70,True,0,0,1
4,Abdoulaye_Doucouré,0,73,0,0,2,36,0,3,60,25.2,3,0,414,0,0,7,2,3,1,10.9,414,48.6,2,2017-08-12T11:30:00Z,12 Aug 12:30,0,0,90,0,0,10,0,0,0,0,11,0,1,0,1207,1,1,0,3,3,35.0,9,0,0,0,50,True,0,0,1


In [24]:
full_data_online_17_18['name'] = full_data_online_17_18['name'].str.split('_', expand=True)[1]

In [25]:
full_data_online_17_18['was_home'] = full_data_online_17_18['was_home'].replace({False:'A', True:'H'})

In [26]:
full_data_online_17_18['join-key'] = full_data_online_17_18['name'] + '-' + full_data_online_17_18['was_home'] + '-' + full_data_online_17_18['transfers_balance'].astype(str)

In [27]:
full_data_sg_17_18.head()

,GW Gameweek,Pts,MPMinutes played,GSGoals scored,AAssists,CSClean sheets,GCGoals conceded,OGOwn goals,PSPenalties saved,PMPenalties missed,YCYellow cards,RCRed cards,SSaves,BBonus,BPSBonus Points System,IInfluence,CCreativity,TThreat,IIICT Index,NT Net Transfers,SB Selected by,£ Value,Name,Team,Position,Total score rank,opposition,home_or_away,team_goals,opposition_goals
0,1,11,85,1,1,0,2,0,0,0,0,0,0,1,26,24.6,2.8,55,8.2,0,874608,9.0,Salah,LIV,MID,1,WAT,A,3,3
1,2,1,29,0,0,0,0,0,0,0,0,0,0,0,0,10.4,12.3,28,5.1,175914,1293309,9.1,Salah,LIV,MID,1,CRY,H,1,0
2,3,11,90,1,1,1,0,0,0,0,0,0,0,0,39,70.4,25.3,103,19.9,-184736,1158692,9.0,Salah,LIV,MID,1,ARS,H,4,0
3,4,1,45,0,0,0,1,0,0,0,0,0,0,0,4,7.8,13.8,25,4.7,177596,1422941,9.1,Salah,LIV,MID,1,MCI,A,0,5
4,5,10,90,1,0,0,1,0,0,0,0,0,0,3,27,51.2,35.8,87,17.4,122769,1571656,9.1,Salah,LIV,MID,1,BUR,H,1,1


In [28]:
full_data_sg_17_18['join-key'] = full_data_sg_17_18['Name'] + '-' + full_data_sg_17_18['home_or_away'] + '-' + full_data_sg_17_18['NT Net Transfers'].astype(str)

In [29]:
combined = full_data_sg_17_18.merge(full_data_online_17_18, on='join-key', how='outer', suffixes=('_sg', '_online'))

In [30]:
combined[(combined['GW Gameweek'].isnull()) & (combined['Gameweek']==1)].head()

,GW Gameweek,Pts,MPMinutes played,GSGoals scored,AAssists,CSClean sheets,GCGoals conceded,OGOwn goals,PSPenalties saved,PMPenalties missed,YCYellow cards,RCRed cards,SSaves,BBonus,BPSBonus Points System,IInfluence,CCreativity,TThreat,IIICT Index,NT Net Transfers,SB Selected by,£ Value,Name,Team,Position,Total score rank,opposition,home_or_away,team_goals,opposition_goals,join-key,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,Gameweek
20096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lookman-H-0,Lookman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,157.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,157.0,0.0,0.0,2017-08-12T14:00:00Z,12 Aug 15:00,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8034.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,55.0,H,0.0,0.0,1.0
20097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Miguel del Castillo-A-0,San Miguel del Castillo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,438.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,438.0,0.0,0.0,2017-08-13T15:00:00Z,13 Aug 16:00,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,24951.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,45.0,A,0.0,0.0,1.0
20098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Musa-A-0,Musa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,207.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,207.0,0.0,0.0,2017-08-11T18:45:00Z,11 Aug 19:45,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35587.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,65.0,A,0.0,0.0,1.0
20099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kolarov-A-0,Kolarov,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,243.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,243.0,0.0,0.0,2017-08-12T16:30:00Z,12 Aug 17:30,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3031.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,A,0.0,0.0,1.0
20101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mitrovic-H-0,Mitrovic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,303.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,303.0,0.0,0.0,2017-08-13T12:30:00Z,13 Aug 13:30,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,121294.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,H,0.0,0.0,1.0


In [31]:
combined_left = full_data_online_17_18.merge(full_data_sg_17_18, on='join-key', how='left', suffixes=('_online', '_sg'))

In [32]:
combined_left[(combined_left['GW Gameweek'].isnull())]['name'].sort_values().unique()

array(['Aarons', 'Afobe', 'Agudo Durán', 'Amrabat', 'Anderson', 'Ayew',
       'Barrow', 'Batshuayi', 'Behrami', 'Berghuis', 'Besic', 'Boga',
       'Borges Da Silva', 'Bunn', 'Burke', 'Carter-Vickers',
       'Castro Pereira', 'Clasie', 'Colback', 'Cook', 'Coquelin',
       'Coutinho', 'Cranie', 'Da Silva Costa', 'Debuchy', 'Feghouli',
       'Femenía Far', 'Flanagan', 'Fletcher', 'Folivi', 'Fonte', 'Fulton',
       'Gouffran', 'Gradel', 'Grujic', 'Gámez Duarte', 'Hanley',
       'Hernández Balcázar', 'Hudson', 'Ibrahimovic', 'Imbula',
       'Izquierdo Mena', 'Janssen', 'Jenkinson', 'Jorgensen', 'Kaikai',
       'Ki', 'Kingsley', 'Kolarov', 'Krkic Perez', 'Ladapo', 'Lee',
       'Lolley', 'Lookman', 'Lucas', 'Luiz Moreira Marinho', 'Luiz Rosa',
       'Mahoney', 'Makasi', 'Martinez', 'Martinez Lopez', 'Mason',
       'Mato Sanmartín', 'McBurnie', 'Melbourne', 'Mendy', 'Mesa',
       'Mirallas', 'Mitrovic', 'Montero', 'Muniesa', 'Murphy', 'Musa',
       'Musonda', 'Mutch', 'Naval Cost

In [33]:
combined[combined['name'] == 'Højbjerg']

,GW Gameweek,Pts,MPMinutes played,GSGoals scored,AAssists,CSClean sheets,GCGoals conceded,OGOwn goals,PSPenalties saved,PMPenalties missed,YCYellow cards,RCRed cards,SSaves,BBonus,BPSBonus Points System,IInfluence,CCreativity,TThreat,IIICT Index,NT Net Transfers,SB Selected by,£ Value,Name,Team,Position,Total score rank,opposition,home_or_away,team_goals,opposition_goals,join-key,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,Gameweek
12119,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13724.0,4.5,Højbjerg,SOU,MID,315.0,SWA,H,0.0,0.0,Højbjerg-H-0,Højbjerg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,321.0,0.0,0.0,2017-08-12T14:00:00Z,12 Aug 15:00,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13724.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,H,0.0,0.0,1.0
12120,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1400.0,13574.0,4.5,Højbjerg,SOU,MID,315.0,WHU,H,3.0,2.0,Højbjerg-H--1400,Højbjerg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,832.0,0.0,0.0,2017-08-19T14:00:00Z,19 Aug 15:00,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,13574.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,-1400.0,359.0,1759.0,45.0,H,0.0,0.0,2.0
12121,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1511.0,12334.0,4.5,Højbjerg,SOU,MID,315.0,HUD,A,0.0,0.0,Højbjerg-A--1511,Højbjerg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,1348.0,0.0,0.0,2017-08-26T14:00:00Z,26 Aug 15:00,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,12334.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1511.0,316.0,1827.0,45.0,A,0.0,0.0,3.0
12122,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2098.0,16171.0,4.4,Højbjerg,SOU,MID,315.0,WAT,H,0.0,2.0,Højbjerg-H-2098,Højbjerg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,1879.0,0.0,0.0,2017-09-09T14:00:00Z,09 Sep 15:00,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,16171.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2098.0,5600.0,3502.0,44.0,H,0.0,0.0,4.0
12123,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-244.0,16353.0,4.4,Højbjerg,SOU,MID,315.0,CRY,A,1.0,0.0,Højbjerg-A--244,Højbjerg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,2423.0,0.0,0.0,2017-09-16T11:30:00Z,16 Sep 12:30,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,16353.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-244.0,1907.0,2151.0,44.0,A,0.0,0.0,5.0
12124,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1345.0,15168.0,4.4,Højbjerg,SOU,MID,315.0,MUN,H,0.0,1.0,Højbjerg-H--1345,Højbjerg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321.0,0.0,0.0,55.0,0.0,0.0,0.0,0.0,2969.0,0.0,0.0,2017-09-23T14:00:00Z,23 Sep 15:00,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,15168.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1345.0,889.0,2234.0,44.0,H,0.0,0.0,6.0
12125,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,15468.0,4.4,Højbjerg,SOU,MID,315.0,STK,A,1.0,2.0,Højbjerg-A-91,Højbjerg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321.0,0.0,0.0,65.0,0.0,0.0,0.0,0.0,3519.0,0.0,0.0,2017-09-30T14:00:00Z,30 Sep 15:00,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,15468.0,0.0,0.0,0.0,1.0,2.

In [34]:
full_data_sg_17_18.groupby('GW Gameweek').count()['Pts']

GW Gameweek
1     434
2     436
3     448
4     461
5     463
6     465
7     468
8     471
9     473
10    475
11    477
12    479
13    480
14    481
15    483
16    485
17    486
18    487
19    490
20    491
21    442
22    545
23    502
24    507
25    516
26    528
27    530
28    533
29    536
30    537
31    225
32    540
33    544
34    757
35    336
36    547
37    865
38    550
Name: Pts, dtype: int64

Much fewer rows per gameweek in SG dataset

In [35]:
full_data_online_17_18.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,Gameweek,join-key
0,Cresswell,0,10,0,0,0,3,0,0,9,0.6,0,0,443,0,0,9,0,2,0,1.9,443,0.4,0,2017-08-13T15:00:00Z,13 Aug 16:00,0,0,9,0,0,12,0,0,0,0,0,0,1,0,25136,0,0,0,0,4,18.0,0,0,0,0,50,A,0,0,1,Cresswell-A-0
1,Lennon,0,0,0,0,0,0,0,0,0,0.0,0,0,153,0,0,5,0,0,0,0.0,153,0.0,0,2017-08-12T14:00:00Z,12 Aug 15:00,0,0,0,0,0,15,0,0,0,0,0,0,1,0,4681,0,0,0,0,1,0.0,0,0,0,0,55,H,0,0,1,Lennon-H-0
2,Mooy,1,52,0,0,0,22,1,5,41,46.9,0,0,172,0,0,4,0,0,0,8.7,172,40.2,3,2017-08-12T14:00:00Z,12 Aug 15:00,0,0,90,0,1,6,0,0,0,0,5,0,1,0,59955,2,0,0,3,0,0.0,6,0,0,0,55,A,0,0,1,Mooy-A-0
3,Ramsey,0,11,0,1,0,16,0,0,8,11.2,0,0,18,0,0,1,0,0,1,6.7,18,29.6,1,2017-08-11T18:45:00Z,11 Aug 19:45,0,0,23,0,0,9,0,0,0,0,2,0,1,0,33792,0,0,3,3,4,26.0,6,0,0,0,70,H,0,0,1,Ramsey-H-0
4,Doucouré,0,73,0,0,2,36,0,3,60,25.2,3,0,414,0,0,7,2,3,1,10.9,414,48.6,2,2017-08-12T11:30:00Z,12 Aug 12:30,0,0,90,0,0,10,0,0,0,0,11,0,1,0,1207,1,1,0,3,3,35.0,9,0,0,0,50,H,0,0,1,Doucouré-H-0


In [36]:
sg_salah = full_data_sg_17_18[full_data_sg_17_18['Name'] == 'Salah']
online_salah = full_data_online_17_18[full_data_online_17_18['name'] == 'Mohamed_Salah']

In [37]:
sg_salah.columns

Index(['GW Gameweek', 'Pts', 'MPMinutes played', 'GSGoals scored', 'AAssists',
       'CSClean sheets', 'GCGoals conceded', 'OGOwn goals',
       'PSPenalties saved', 'PMPenalties missed', 'YCYellow cards',
       'RCRed cards', 'SSaves', 'BBonus', 'BPSBonus Points System',
       'IInfluence', 'CCreativity', 'TThreat', 'IIICT Index',
       'NT Net Transfers', 'SB Selected by', '£ Value', 'Name', 'Team',
       'Position', 'Total score rank', 'opposition', 'home_or_away',
       'team_goals', 'opposition_goals', 'join-key'],
      dtype='object')

In [ ]:
combined.head()

In [ ]:
full_data_sg_17_18[full_data_sg_17_18['Name'] == 'Salah']

## Check 2018_19 data

In [39]:
from src.features.clean_scraped_data import clean_raw_scraped_data

In [41]:
sg_2018_19 = clean_raw_scraped_data('data/raw/2018_19_current_season_data.csv')
sg_2018_19.head()

,GW Gameweek,Pts,MPMinutes played,GSGoals scored,AAssists,CSClean sheets,GCGoals conceded,OGOwn goals,PSPenalties saved,PMPenalties missed,YCYellow cards,RCRed cards,SSaves,BBonus,BPSBonus Points System,IInfluence,CCreativity,TThreat,IIICT Index,NT Net Transfers,SB Selected by,£ Value,Name,Team,Position,opposition,home_or_away,team_goals,opposition_goals
0,1,8,86,1,0,1,0,0,0,0,0,0,0,0,20.0,47.6,29.0,92.0,16.9,0.0,2478960.0,13.0,Salah,LIV,MID,WHU,H,4,0
1,2,9,90,0,2,1,0,0,0,0,0,0,0,0,22.0,30.6,41.5,81.0,15.3,38860.0,2799491.0,13.0,Salah,LIV,MID,CRY,A,2,0
2,3,8,90,1,0,1,0,0,0,0,0,0,0,0,28.0,49.0,72.0,82.0,20.3,5370.0,2892383.0,13.0,Salah,LIV,MID,BHA,H,1,0
3,4,2,70,0,0,0,1,0,0,0,0,0,0,0,7.0,5.0,12.5,23.0,4.1,32547.0,2995784.0,13.0,Salah,LIV,MID,LEI,A,2,1
4,5,2,90,0,0,0,1,0,0,0,0,0,0,0,7.0,16.0,31.8,74.0,12.2,-261759.0,2754458.0,13.0,Salah,LIV,MID,TOT,A,2,1


In [45]:
sg_2018_19['join-key'] = sg_2018_19['Name'] + '-' + sg_2018_19['home_or_away'] + '-' + sg_2018_19['NT Net Transfers'].astype(str)

In [44]:
full_data_online_18_19 = pd.read_csv('data/external/online_fpl_data/2018_19/merged_gw.csv', encoding = "ISO-8859-1")

In [47]:
full_data_online_18_19['name'] = full_data_online_18_19['name'].str.split('_', expand=True)[1]

In [48]:
full_data_online_18_19['was_home'] = full_data_online_18_19['was_home'].replace({False:'A', True:'H'})

In [69]:
full_data_online_18_19['transfers_balance'] = full_data_online_18_19['transfers_balance'].astype(float)

In [70]:
full_data_online_18_19['join-key'] = full_data_online_18_19['name'] + '-' + full_data_online_18_19['was_home'] + '-' + full_data_online_18_19['transfers_balance'].astype(str)


In [71]:
full_data_online_18_19.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW,join-key
0,Cresswell,0,0,0,0,0,0,0,0,0,0.0,0,0,402,0,0,5,0,0,0,0.0,402,0.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,0,0,0,12,0,0,0,0,0,0,1,0,103396,0,0,0,0,4,0.0,0,0.0,0,0,55,A,0,0,1,Cresswell-A-0.0
1,Lennon,0,22,0,1,0,6,1,1,17,12.3,0,0,83,0,0,8,1,0,0,3.9,83,10.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,90,0,1,16,0,0,0,0,2,0,1,0,15138,1,2,0,0,0,17.0,3,0.0,0,0,50,A,0,0,1,Lennon-A-0.0
2,Mooy,0,51,0,0,0,24,0,2,40,18.2,1,0,199,0,0,4,1,3,0,3.8,199,20.2,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,6,0,0,0,0,11,0,1,0,192110,1,6,0,3,0,0.0,2,0.0,0,0,55,H,0,0,1,Mooy-H-0.0
3,Ramsey,0,11,0,0,0,7,0,0,7,10.8,1,0,14,0,0,1,0,1,0,2.9,14,9.4,1,2018-08-12T15:00:00Z,12 Aug 16:00,0,0,53,2,0,13,0,0,0,0,1,0,1,0,60423,0,2,0,2,0,9.0,1,0.0,0,0,75,H,0,0,1,Ramsey-H-0.0
4,Wan-Bissaka,1,29,1,0,3,38,1,11,19,14.0,2,0,145,0,0,3,0,0,0,6.0,145,46.0,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,9,0,0,0,0,7,0,1,0,652304,2,0,0,2,0,0.0,12,0.0,0,0,40,A,0,0,1,Wan-Bissaka-A-0.0


In [72]:
len(full_data_online_18_19)

21790

In [73]:
len(sg_2018_19)

18654

In [74]:
combined = full_data_online_18_19.merge(sg_2018_19, how='left', on='join-key')

In [82]:
sg_2018_19['Name'].sort_values().unique()

array(['Adam Smith', 'Adrián', 'Agüero', 'Aké', 'Albrighton',
       'Alderweireld', 'Alexander-Arnold', 'Alisson', 'Alli', 'Almirón',
       'Alonso', 'Amartey', 'Amos', 'Ampadu', 'Andone', 'André Gomes',
       'Antonio', 'Armstrong', 'Arnautovic', 'Arter', 'Atsu',
       'Aubameyang', 'Aurier', 'Austin', 'Ayew', 'Ayité', 'Azpilicueta',
       'Babel', 'Bacuna', 'Bailly', 'Baines', 'Balbuena', 'Balogun',
       'Bamba', 'Bardsley', 'Barkley', 'Barnes', 'Barreca', 'Batshuayi',
       'Bednarek', 'Begovic', 'Bellerín', 'Bennett', 'Benson', 'Benteke',
       'Bernard', 'Bernardo', 'Bernardo Silva', 'Bertrand', 'Bettinelli',
       'Billing', 'Bissouma', 'Boly', 'Bong', 'Boruc', 'Brady', 'Bravo',
       'Brian Murphy', 'Britos', 'Brooks', 'Bruno', 'Bryan', 'Burn',
       'Butcher', 'Button', 'Caballero', 'Cahill', 'Cairney',
       'Calvert-Lewin', 'Camacho', 'Camarasa', 'Capoue', 'Carlos Sánchez',
       'Carroll', 'Cass', 'Cathcart', 'Cavaleiro', 'Cech', 'Chalobah',
       'Chamberlain

In [84]:
combined[combined['GW Gameweek'].isnull()]['name'].sort_values().unique()

array(['Abraham', 'Agosto Ramírez', 'Anício Caldeira Duarte',
       'Arrizabalaga', 'Bakayoko', 'Baldock', 'Baningime',
       'Barbosa Pereira', 'Batth', 'Benalouane', 'Benkovic', 'Blind',
       'Bogle', 'Bolasie', 'Bonatini', 'Borges Da Silva', 'Boufal',
       'Browning', 'Byram', 'Carrillo', 'Castro Otto', 'Cook',
       'Costa Silva', 'Courtois', 'Cunninghamm', 'Dalot Teixeira',
       'Davis', 'De Gea', 'Defoe', 'Dembélé', 'Diabaté', 'Diaz',
       'Douglas', 'Dowell', 'Enobakhare', 'Fellaini', 'Femenía Far',
       'Fernandes', 'Fernandes da Silva Junior', 'Frello Filho',
       'Fàbregas', 'Gabbiadini', 'Gayle', 'Gonçalves Miranda', 'Halford',
       'Hause', 'Hemed', 'Hernández Balcázar', 'Hoedt', 'Holgate',
       'Hugill', 'Iborra', 'Ince', 'Izquierdo Mena', 'Jach', 'Johansen',
       'Jorgensen', 'Kaboul', 'Kaikai', 'Kamara', 'Karius', 'Ki',
       'Klaassen', 'Klavan', 'Knight', 'Koiki', 'Luiz Moreira Marinho',
       'Luiz Rosa', 'Malone', 'Martina', 'Martínez', 'Mato S

In [68]:
sg_2018_19[sg_2018_19['Name'] == 'Salah']

,GW Gameweek,Pts,MPMinutes played,GSGoals scored,AAssists,CSClean sheets,GCGoals conceded,OGOwn goals,PSPenalties saved,PMPenalties missed,YCYellow cards,RCRed cards,SSaves,BBonus,BPSBonus Points System,IInfluence,CCreativity,TThreat,IIICT Index,NT Net Transfers,SB Selected by,£ Value,Name,Team,Position,opposition,home_or_away,team_goals,opposition_goals,join-key
0,1,8,86,1,0,1,0,0,0,0,0,0,0,0,20.0,47.6,29.0,92.0,16.9,0.0,2478960.0,13.0,Salah,LIV,MID,WHU,H,4,0,Salah-H-0.0
1,2,9,90,0,2,1,0,0,0,0,0,0,0,0,22.0,30.6,41.5,81.0,15.3,38860.0,2799491.0,13.0,Salah,LIV,MID,CRY,A,2,0,Salah-A-38860.0
2,3,8,90,1,0,1,0,0,0,0,0,0,0,0,28.0,49.0,72.0,82.0,20.3,5370.0,2892383.0,13.0,Salah,LIV,MID,BHA,H,1,0,Salah-H-5370.0
3,4,2,70,0,0,0,1,0,0,0,0,0,0,0,7.0,5.0,12.5,23.0,4.1,32547.0,2995784.0,13.0,Salah,LIV,MID,LEI,A,2,1,Salah-A-32547.0
4,5,2,90,0,0,0,1,0,0,0,0,0,0,0,7.0,16.0,31.8,74.0,12.2,-261759.0,2754458.0,13.0,Salah,LIV,MID,TOT,A,2,1,Salah-A--261759.0
5,6,8,90,1,0,1,0,0,0,0,0,0,0,0,18.0,34.0,2.9,60.0,9.7,-352832.0,2419608.0,13.0,Salah,LIV,MID,SOU,H,3,0,Salah-H--352832.0
6,7,2,65,0,0,0,1,0,0,0,0,0,0,0,8.0,9.4,25.7,40.0,7.5,-106042.0,2330574.0,12.9,Salah,LIV,MID,CHE,A,1,1,Salah-A--106042.0
7,8,3,90,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.9,31.0,3.3,-338250.0,1992452.0,12.9,Salah,LIV,MID,MCI,H,0,0,Salah-H--338250.0
8,9,8,90,1,0,1,0,0,0,0,0,0,0,0,26.0,43.0,12.5,76.0,13.2,-321994.0,1681931.0,12.8,Salah,LIV,MID,HUD,A,1,0,Salah-A--321994.0
9,10,15,90,1,2,0,1,0,0,0,0,0,0,2,47.0,72.4,45.8,56.0,17.4,251072.0,1978411.0,12.9,Salah,LIV,MID,CAR,H,4,1,Salah-H-251072.0


In [59]:
full_data_online_18_19[full_data_online_18_19['name'] == 'Salah']

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW,join-key
366,Salah,0,26,0,1,0,20,1,1,22,29.0,1,0,253,0,0,5,0,0,1,16.9,253,47.6,2,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,86,1,0,19,0,0,0,0,3,0,1,0,2478960,6,0,1,0,4,92.0,8,0,0,0,130,H,1,0,1,Salah-H-0
892,Salah,2,19,1,2,0,22,1,0,13,41.5,3,0,253,0,0,15,1,0,0,15.3,778,30.6,4,2018-08-20T19:00:00Z,20 Aug 20:00,0,0,90,2,0,7,0,0,0,0,5,0,2,0,2799491,3,0,1,2,0,81.0,9,38860,69743,30883,130,A,0,0,2,Salah-A-38860
1422,Salah,0,40,0,0,0,28,1,0,29,72.0,2,0,253,0,0,25,1,0,1,20.3,1305,49.0,6,2018-08-25T16:30:00Z,25 Aug 17:30,0,0,90,0,0,3,0,0,0,0,1,0,3,0,2892383,8,1,2,0,1,82.0,8,5370,80831,75461,130,H,1,0,3,Salah-H-5370
1956,Salah,0,22,1,1,0,7,0,0,18,12.5,2,0,253,0,0,37,0,1,0,4.1,1836,5.0,1,2018-09-01T11:30:00Z,01 Sep 12:30,0,0,70,2,0,11,0,0,0,0,6,0,4,0,2995784,1,0,1,2,1,23.0,2,32547,79082,46535,130,A,0,0,4,Salah-A-32547
2493,Salah,0,14,0,1,0,7,0,0,11,31.8,3,0,253,0,0,48,1,1,0,12.2,2373,16.0,3,2018-09-15T11:30:00Z,15 Sep 12:30,0,0,90,0,0,17,0,0,0,0,4,0,5,0,2754458,2,0,0,2,1,74.0,2,-261759,35679,297438,130,A,0,0,5,Salah-A--261759
3031,Salah,0,25,0,0,0,18,1,0,15,2.9,3,0,253,0,0,58,1,0,1,9.7,2910,34.0,0,2018-09-22T14:00:00Z,22 Sep 15:00,0,0,90,3,0,16,0,0,0,0,3,0,6,0,2419608,5,0,1,0,3,60.0,8,-352832,21189,374021,130,H,0,0,6,Salah-H--352832
3570,Salah,0,31,0,1,0,8,0,0,23,25.7,3,0,253,0,0,64,1,1,0,7.5,3448,9.4,1,2018-09-29T16:30:00Z,29 Sep 17:30,0,0,65,1,1,6,0,0,0,0,4,0,7,0,2330574,1,1,2,1,1,40.0,2,-106042,22128,128170,129,A,0,0,7,Salah-A--106042
4110,Salah,0,16,0,0,0,1,1,1,11,1.9,0,0,253,0,0,76,1,0,0,3.3,3987,0.0,0,2018-10-07T15:30:00Z,07 Oct 16:30,0,0,90,1,0,13,0,0,0,0,3,0,8,0,1992452,2,0,2,0,0,31.0,3,-338250,7940,346190,129,H,0,0,8,Salah-H--338250
4655,Salah,0,44,0,0,0,26,1,1,34,12.5,2,0,253,0,0,86,0,0,1,13.2,4529,43.0,0,2018-10-20T16:30:00Z,20 Oct 17:30,0,0,90,0,1,10,0,0,0,0,5,0,9,0,1681931,6,0,1,1,0,76.0,8,-321994,34581,356575,128,A,1,0,9,Salah-A--321994
5203,Salah,2,50,2,0,2,47,0,0,43,45.8,0,0,253,0,0,96,3,1,1,17.4,5075,72.4,3,2018-10-27T14:00:00Z,27 Oct 15:00,0,0,90,0,1,5,0,0,0,0,2,0,10,0,1978411,5,0,1,1,4,56.0,15,251072,271983,20911,129,H,0,0,10,Salah-H-251072


- Some proof that numbers allign across both sources